In [1]:
import pandas as pd
import numpy as np
import os
import subprocess
import sys
from pathlib import Path
from datetime import datetime
from datetime import timedelta
from multiprocessing import Process, Pool
from dateutil.relativedelta import *
import dateutil.parser

In [2]:
GPS_info = pd.read_csv("IGSNetwork.csv")
df_stations = GPS_info["#StationName"]

In [3]:

pre_path = "D://dataframe"

headers = ['dates', 'tec']
dtypes = {'dates': 'str', 'tec': 'float'}
parse_dates = ['dates']

In [4]:
def read_csv_and_erase(station):
    headers = ['dates', 'tec']
    dtypes = {'dates': 'str', 'tec': 'float'}
    parse_dates = ['dates']
    path ="{pre_path}/{station}.csv.xz".format(pre_path = pre_path, station = station)
    if os.path.isfile(path): 
        df_output = pd.read_csv(path, index_col='dates')
    else:
        df_output = pd.DataFrame()
    for file in Path(".").glob("*{station}*".format(station=station)):
        with file as f:
            current_df_output = pd.read_csv(f, header=None,  comment='#',sep = ' ',names=headers, dtype=dtypes, parse_dates=parse_dates)
            current_df_output = current_df_output.set_index("dates")
            df_output = pd.concat([df_output, current_df_output])
        df_output.to_csv(path , compression = 'xz')
        file.unlink()

In [5]:
def get_tec_station(start_day, end_day, station):
    all_days = [start_day+timedelta(days=x) for x in range((end_day-start_day).days)]
    for day in all_days:
        print(str(station) + " -- " + str(day.strftime('%Y-%m-%d')))
        result = subprocess.run(["ionolabtec.exe", str(station), str(day.strftime('%Y-%m-%d'))], capture_output=True)
        if "does not exist.\\r\\n" in str(result.stdout):
            print(str(station) + " is not valid")
        read_csv_and_erase(str(station))
    return 0

In [ ]:
import concurrent.futures
import urllib.request

stations =  list(df_stations[0:10])

def f(station):
    start_day = datetime(2020, 1, 1).date()
    end_day = datetime(2020, 1, 5).date()
    get_tec_station(start_day,end_day, station)

with concurrent.futures.ThreadPoolExecutor(max_workers = 10) as executor:

    future_to_url = {executor.submit(f, station): station for station in stations}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

ABMF00GLP -- 2020-01-01
ABPO00MDG -- 2020-01-01
AC2300USA -- 2020-01-01
AC2400USA -- 2020-01-01
ACRG00GHA -- 2020-01-01
ACSO00USA -- 2020-01-01
ADIS00ETH -- 2020-01-01
AGGO00ARG -- 2020-01-01
AIRA00JPN -- 2020-01-01
AJAC00FRA -- 2020-01-01
ACRG00GHA is not valid
ACSO00USA is not valid
ACRG00GHA -- 2020-01-02
ACSO00USA -- 2020-01-02
ADIS00ETH is not valid
ADIS00ETH -- 2020-01-02
AC2300USA is not valid
AC2300USA -- 2020-01-02
AJAC00FRA is not valid
AJAC00FRA -- 2020-01-02
AC2400USA is not valid
AC2400USA -- 2020-01-02
AIRA00JPN -- 2020-01-02
ABPO00MDG -- 2020-01-02
AGGO00ARG -- 2020-01-02
ABMF00GLP -- 2020-01-02
AJAC00FRA is not valid
AJAC00FRA -- 2020-01-03
ACSO00USA is not valid
ACSO00USA -- 2020-01-03
ADIS00ETH is not valid
ADIS00ETH -- 2020-01-03
ACRG00GHA is not valid
ACRG00GHA -- 2020-01-03
AC2300USA is not valid
AC2300USA -- 2020-01-03
AC2400USA is not valid
AC2400USA -- 2020-01-03
AIRA00JPN -- 2020-01-03
AGGO00ARG -- 2020-01-03
ABPO00MDG -- 2020-01-03
ABMF00GLP -- 2020-01-03
AJAC